<a href="https://colab.research.google.com/github/sameersam9519/1st-jan-Nifty/blob/master/Copy_of_SMA_Strategy_Alice_Blue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install alice-blue

     |████████████████████████████████| 133kB 19.4MB/s 
     |████████████████████████████████| 204kB 27.0MB/s 
  Created wheel for protlib: filename=protlib-1.4-cp36-none-any.whl size=16509 sha256=bb8a6866eb4ba7b715535f0d8faaca2d79a7e71305210f54c109e72ddee8fc5d
  Stored in directory: /root/.cache/pip/wheels/8a/63/00/ab8216d236452dc3e62006da7e843600104daa3076f2ce18a8
Successfully built protlib


In [ ]:
import logging
import datetime
import statistics
from time import sleep
from alice_blue import *

# Config
username = 'username'
password = 'password'
api_secret = 'api_secret'
twoFA = 'twoFA'
EMA_CROSS_SCRIP = 'INFY'
logging.basicConfig(level=logging.DEBUG)        # Optional for getting debug messages.
# Config

ltp = 0
socket_opened = False
alice = None
def event_handler_quote_update(message):
    global ltp
    ltp = message['ltp']

def open_callback():
    global socket_opened
    socket_opened = True

def buy_signal(ins_scrip):
    global alice
    alice.place_order(transaction_type = TransactionType.Buy,
                         instrument = ins_scrip,
                         quantity = 1,
                         order_type = OrderType.Market,
                         product_type = ProductType.Intraday,
                         price = 0.0,
                         trigger_price = None,
                         stop_loss = None,
                         square_off = None,
                         trailing_sl = None,
                         is_amo = False)

def sell_signal(ins_scrip):
    global alice
    alice.place_order(transaction_type = TransactionType.Sell,
                         instrument = ins_scrip,
                         quantity = 1,
                         order_type = OrderType.Market,
                         product_type = ProductType.Intraday,
                         price = 0.0,
                         trigger_price = None,
                         stop_loss = None,
                         square_off = None,
                         trailing_sl = None,
                         is_amo = False)
    
def main():
    global socket_opened
    global alice
    global username
    global password
    global twoFA
    global api_secret
    global EMA_CROSS_SCRIP
    minute_close = []
    access_token =  AliceBlue.login_and_get_access_token(username=username, password=password, twoFA=twoFA,  api_secret=api_secret)
    alice = AliceBlue(username=username, password=password, access_token=access_token, master_contracts_to_download=['NSE'])
    
    print(alice.get_balance()) # get balance / margin limits
    print(alice.get_profile()) # get profile
    print(alice.get_daywise_positions()) # get daywise positions
    print(alice.get_netwise_positions()) # get netwise positions
    print(alice.get_holding_positions()) # get holding positions
    
    ins_scrip = alice.get_instrument_by_symbol('NSE', EMA_CROSS_SCRIP)
    
    socket_opened = False
    alice.start_websocket(subscribe_callback=event_handler_quote_update,
                          socket_open_callback=open_callback,
                          run_in_background=True)
    while(socket_opened==False):    # wait till socket open & then subscribe
        pass
    alice.subscribe(ins_scrip, LiveFeedType.COMPACT)
    
    while True:
        if(datetime.datetime.now().second == 0):
            minute_close.append(ltp)
            print(ltp) #Just added this line so that you know that it is fetching the LTP.
            if(len(minute_close) > 20):
                sma_5 = statistics.mean(minute_close[-5:])
                sma_20 = statistics.mean(minute_close[-20:])
                if(sma_5 > sma_20):
                    buy_signal(ins_scrip)
                elif(sma_5 < sma_20):
                    sell_signal(ins_scrip)
            sleep(1)
        sleep(0.2)  # sleep for 200ms
    
if(__name__ == '__main__'):
    main()


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ant.aliceblueonline.com:443
DEBUG:urllib3.connectionpool:https://ant.aliceblueonline.com:443 "GET /oauth2/auth?response_type=code&state=test_state&client_id=WBK268&redirect_uri=https://ant.aliceblueonline.com/plugin/callback HTTP/1.1" 302 115
DEBUG:urllib3.connectionpool:https://ant.aliceblueonline.com:443 "GET /oauth/login?login_challenge=4e09201f1c8a4b1ea6dbcbb5bc19ae5b HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://ant.aliceblueonline.com:443 "POST /oauth/login?login_challenge=4e09201f1c8a4b1ea6dbcbb5bc19ae5b HTTP/1.1" 302 371
DEBUG:urllib3.connectionpool:https://ant.aliceblueonline.com:443 "GET /oauth/twofa?login_challenge=4e09201f1c8a4b1ea6dbcbb5bc19ae5b&login_id=WBK268&question_ids=27%7C37&questions=With+which+company+did+you+start+your+career%3F+%28e.g.+TATA%2C+Infosys%2C+Etc.%29%7CWhich+web-site+you+created+your+first+e-mail+account%3F+%28e.g.++Rediff%2C+Sify%2C+Office%2C+etc%29 HTTP/1.1" 200 None
INFO:roo

Exception: ignored